In [1]:
import os
# import plotly.graph_objects as go
from tqdm import tqdm
from gym import envs
import argparse
import numpy as np
from bigmdp.data.buffer import StandardBuffer,ReplayBuffer, gather_data_in_buffer, get_iter_indexes
from bigmdp.mdp.MDP_GPU import FullMDP
from bigmdp.utils.utils_eval import evaluate_on_env
from bigmdp.mdp.agent import SimpleAgent
from copy import deepcopy as cpy
from os import path
from arg_def import * 
import gym
from sklearn.neighbors import KDTree
from IPython import display
import torch
import time
import copy

In [2]:
parser, ArgumentDict = get_argument_parser()
options = "--env_name CartPole-v0 --action_repeat 1 --max_episode_length 200 --MAX_S_COUNT 15000 --MAX_NS_COUNT 5"
args = parser.parse_args(options.split(" "))


In [3]:
from image_wrappers import *
env = gym.make(args.env_name)

In [4]:
train_buffer = ReplayBuffer(state_dim = env.observation_space.shape[0],
                           is_atari= False, 
                           atari_preprocessing= None, 
                           batch_size=32, 
                           buffer_size=20000,
                           device="cpu")



In [5]:
train_buffer, info = gather_data_in_buffer(train_buffer, env,policy = lambda s:np.random.randint(2), episode_count=99999, frame_count=10)

Average Reward of collected trajectories:15.0


In [6]:
class DummyNet():
    def __init__(self, sim, add_noise=False):
        self.simulator = sim
        self.use_cuda = False
#         self.add_noise = add_noise

    def encode_single(self, o):
        return tuple(o)

    def encode_batch(self, o_batch):
        return [tuple(o) for o in o_batch]

    def predict_single_transition(self, o, a):
        return False

    def predict_batch_transition(self, o_batch, a_batch):
        return False


In [32]:
def build_mdp(self, train_buffer):

    print("Step 1 (Parse Transitions):  Running")
    st = time.time()

    _batch_size = 256
    start_end_indexes = get_iter_indexes(len(train_buffer), _batch_size)
    for start_i, end_i in tqdm(start_end_indexes):
        batch = train_buffer.sample_indices(list(range(start_i, end_i)))
        batch_ob, batch_a, batch_ob_prime, batch_r, batch_nd = batch
        batch_d = 1 - batch_nd
        self.batch_parse(batch_ob.numpy(), batch_a.numpy().squeeze(), batch_ob_prime.numpy(),
                         batch_r.numpy().squeeze(), batch_d.numpy().squeeze())

    print("Step 1 [Parse Transitions]:  Complete,  Time Elapsed: {}\n\n".format(time.time() - st))

    print("Step 2 [Seed Seen Transitions + Unknown (s,a) pairs]:  Running")
    st = time.time()
    self.commit_seen_transitions()
    print("Step 2 (Commit Seen Transitions):  Complete,  Time Elapsed: {} \n\n".format(time.time() - st))

    print("Step 3 [Commit all Transitions]:  Running")
    st = time.time()
    commit_predicted_transitions(self,verbose=True)
    print("Step 3 (Commit UnSeen Transitions):  Complete,  Time Elapsed: {}".format(time.time() - st))

#     print("Step 4 [Solve MDP]:  Running")
#     st = time.time()
#     self.solve_mdp()
#     print("% of missing transitions",
#           self.mdp_T.unknown_state_action_count / (len(self.mdp_T.tD) * len(self.mdp_T.A)))
#     self.seed_policies(smoothing=False, soft_q=False)
#     print("Step 4 [Solve MDP]:  Complete,  Time Elapsed: {}\n\n".format(time.time() - st))

In [37]:

def commit_predicted_transitions(self, verbose=False):
    if self.fill_with == "0Q_src-KNN":
        iterator_ = self.to_commit_sa_pairs.items()
        iterator_ = tqdm(iterator_) if verbose else iterator_

        for sa_pair, seen_flag in iterator_:
            # parse sa_pair
            s_, a_ = sa_pair
            s_i, a_i = self.mdp_T.s2i[s_], self.mdp_T.a2i[a_]

            # get distances
            knn_sa = self.mdp_T._get_knn_hs_kd_with_action_tree((s_, a_), k=self.mdp_T.mdp_build_k)
            knn_sa_normalized = self.mdp_T.get_kernel_probs(knn_sa, delta=self.mdp_T.knn_delta)
            self.dist_to_nn_cache.extend(list(knn_sa.values()))

            # get new transition counts
            tran_counts, reward_counts = defaultdict(init2zero), defaultdict(init2zero)
            for nn_sa, norm_dist in knn_sa_normalized.items():
                nn_s, a = nn_sa
                for nn_ns in list(self.mdp_T.known_tC[nn_s][a].keys()):
                    orig_tr, orig_tc = self.mdp_T.known_tC[nn_s][a][nn_s], self.mdp_T.known_rC[nn_s][a][nn_s]
                    print( (s_, a_), (s_i, a_i), (nn_s, self.mdp_T.s2i[nn_s]),(nn_ns,  self.mdp_T.s2i[nn_ns]), orig_tr, orig_tc)
                    tran_counts[nn_ns] += int(norm_dist * 100 * orig_tc) if self.norm_by_dist else 1
                    reward_counts[nn_ns] += int(
                        norm_dist * 100 * orig_tr) if self.norm_by_dist else orig_tr / orig_tc

            top_k_ns = heapq.nlargest(self.mdp_T.mdp_build_k, tran_counts, key=tran_counts.get)
            tran_counts = {s: tran_counts[s] for s in top_k_ns}  # filter for overflow
            reward_counts = {s: reward_counts[s] for s in top_k_ns}  # filter for overflow
            new_transitions = [(i, ns, tran_counts[ns], reward_counts[ns]) for i, ns in enumerate(tran_counts)]

            # update count matrices
            assert len(new_transitions) == self.mdp_T.mdp_build_k
            for slot, ns, t_count, r_count in new_transitions:
                ns_i = self.mdp_T.s2i[ns]
                self.mdp_T.tranidxMatrix_cpu[a_i, s_i, slot] = ns_i
                self.mdp_T.tranCountMatrix_cpu[a_i, s_i, slot] = t_count
                self.mdp_T.rewardCountMatrix_cpu[a_i, s_i, slot] = r_count

            # update prob matrices
            self.mdp_T.update_prob_matrices(s_i, a_i)

        self.to_commit_sa_pairs = defaultdict(init2zero)
        self.to_commit_transitions = []


In [38]:

empty_MDP = FullMDP(A= list(range(env.action_space.n)),
                    ur=args.unknown_transition_reward,
                    vi_params={"gamma":  args.gamma,
                               "slip_prob": args.slip_probability,
                               "rmax_reward": args.rmax_reward,
                               "rmax_thres": args.rmax_threshold,
                               "balanced_explr": args.balanced_exploration,
                              "rmin":-1000},
                    mdp_params={"weight_transitions": args.weight_transitions,
                            "mdp_build_k":args.mdp_build_k,
                            "plcy_lift_k": 1,
                            "knn_delta" : args.knn_delta,
                            "calc_action_vector":False,
                            "penalty_beta":args.penalty_beta,
                            "penalize_unknown_transitions": args.penalty_type != "none"
                            },
                    MAX_S_COUNT= args.MAX_S_COUNT,
                    MAX_NS_COUNT= args.MAX_NS_COUNT,
                    default_mode=args.def_device)

myAgent =  SimpleAgent(empty_MDP, net = DummyNet(None),  fill_with = args.fill_with,
                       sample_neighbors = False, pred_error_threshold = args.within_radius,
                       penalty_type=args.penalty_type, penalty_beta = args.penalty_beta,
                       abstraction_flag=False, normalize_by_distance= args.normalize_by_distance)

mdp_id = "L[{}]-E[{}]-B[{}]-Fill[{}_K{}]-Penalty[{}_{}]".format(args.latent_type, args.encoder_type, args.bottleneck_size, 
                                                          args.fill_with, args.mdp_build_k, args.penalty_type, args.penalty_beta)

# myAgent.build_mdp(train_buffer)
build_mdp(myAgent,train_buffer)


  0%|          | 0/30 [00:00<?, ?it/s]

Step 1 (Parse Transitions):  Running
Step 1 [Parse Transitions]:  Complete,  Time Elapsed: 0.0022797584533691406


Step 2 [Seed Seen Transitions + Unknown (s,a) pairs]:  Running
Len of to commit transitions 14
ABstraction Faldg False
Len of to commit sa pairs 30
Step 2 (Commit Seen Transitions):  Complete,  Time Elapsed: 0.0026102066040039062 


Step 3 [Commit all Transitions]:  Running
((-0.026770264, 0.013590637, -0.03349538, 0.03398315), 0) (2, 0) ((-0.026498452, 0.20917651, -0.032815717, -0.2690769), 3) ((-0.022314921, 0.014537863, -0.038197257, 0.013077795), 4) 0 0


ZeroDivisionError: division by zero

In [11]:
myAgent.mdp_T.refresh_cache_dicts()

IndexError: index 5 is out of bounds for axis 2 with size 5

In [24]:
def refresh_cache_dicts(self):
    self.tD = defaultdict(init2zero_def_def_dict)  # Transition Probabilities
    self.rD = defaultdict(init2zero_def_def_dict)  # Transition Probabilities

    for s, s_i in self.s2i.items():
        for a in self.A:
            for ns_slot, ns_i in enumerate(self.tranidxMatrix_cpu[self.a2i[a], s_i]):
                ns,a_i = self.i2s[ns_i], self.a2i[a]
                if self.tranProbMatrix_cpu[a_i, s_i, ns_slot] >0:
                    print(f"ns:{ns},ns_i:{ns_i}, a_i:{a_i}, a:{a}, s:{s}, s_i:{s_i} ,  ns_slot:{ns_slot}")
                    self.tD[s][a][ns] = self.tranProbMatrix_cpu[a_i, s_i, ns_slot]
                    self.rD[s][a][ns] = self.rewardMatrix_cpu[a_i, s_i, ns_slot]
                # Transition Probabilities

In [29]:
# refresh_cache_dicts(empty_MDP)
# empty_MDP.tranProbMatrix_cpu

In [30]:
# for s in empty_MDP.tD:
#     for a in empty_MDP.tD[s]:
#         print(empty_MDP.s2i[s],a)
#         for ns in empty_MDP.tD[s][a]:
#             print("\t",empty_MDP.s2i[ns],empty_MDP.tD[s][a][ns])

In [8]:
myAgent.smoothing = True
myAgent.soft_q = False
myAgent.mdp_T.plcy_lift_k = 11
myAgent.mdp_T.smooth_with_seen = False

sum_reward_running = {policy_name:evaluate_on_env(env,  
                                                  policy, 
                                                  eps_count=50,progress_bar=True)[0]
                      for policy_name,policy in myAgent.policies.items()}
print(sum_reward_running)

100%|██████████| 50/50 [00:02<00:00, 18.66it/s]

{'optimal': 200.0, 'random': 20.96, 'eps_optimal': 200.0, 'safe': 200.0}


In [9]:
# evaluate_on_env(env,  myAgent.policies["optimal"], 
#                                                   eps_count=50,progress_bar=True)


In [ ]:
for s in myAgent.mdp_cache:
    for a in myAgent.mdp_cache[s]:
        for ns in myAgent.mdp_cache[s][a]:

In [17]:
a = {1:2, 3:1}
max(a, key=a.get)

1

In [10]:
from pycuda import compiler, gpuarray
from collections import defaultdict
import time
import math as mth
from bigmdp.mdp.kernels import complex_vi_kernel_code_template as vi_kernel_template
import numpy as np
from tqdm import tqdm
from copy import deepcopy as cpy
from collections import deque
import random
from sklearn.neighbors import KDTree
from collections import Counter

In [11]:
e_vD_gpu = gpuarray.to_gpu(np.zeros((100, 1)).astype('float32'))
e_vD_cpu = np.zeros((100, 1)).astype('float32')

In [12]:
for i in tqdm(range(int(1e6))):
    e_vD_cpu[1][0] = 1

100%|██████████| 1000000/1000000 [00:00<00:00, 1442770.47it/s]


In [13]:
for i in tqdm(range(int(1e6))):
    e_vD_gpu[1][0] = 1

  0%|          | 0/1000000 [00:00<?, ?it/s]


TypeError: src must be GPUArray or ndarray

In [36]:
tranCountMatrix_cpu = np.ones((2, 4, 3)).astype('float32')
tranProbMatrix_cpu = np.zeros((2, 4, 3)).astype('float32')
tranCountMatrix_cpu[0,0,-1] = 0

In [37]:
a_i = 0
s_i = 0
tranProbMatrix_cpu[a_i,s_i] = tranCountMatrix_cpu[a_i,s_i]/np.sum(tranCountMatrix_cpu[a_i,s_i])

In [38]:
tranProbMatrix_cpu

array([[[0.5, 0.5, 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ]],

       [[0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ]]], dtype=float32)

In [31]:
tranCountMatrix_cpu

array([[[5., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[5., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]], dtype=float32)